<a href="https://colab.research.google.com/github/CFeenan/SolarCNN/blob/master/CNN_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Upload CSV files
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()
uploaded = files.upload()


Saving Str1_OV_Nov_15.csv to Str1_OV_Nov_15.csv


Saving Str1_MV_Nov_15.csv to Str1_MV_Nov_15.csv


Saving Str1_P_Nov_15.csv to Str1_P_Nov_15.csv


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

# Load all 3 CSVs
mv = pd.read_csv("Str1_MV_Nov_15.csv")
ov = pd.read_csv("Str1_OV_Nov_15.csv")
p = pd.read_csv("Str1_P_Nov_15.csv")

# Define a function to clean each file
def clean_timestamp(df):
    # Parse and split timestamp
    df['Time'] = pd.to_datetime(df['Time'], format="%d/%m/%Y %H:%M")
    df['Date'] = df['Time'].dt.date
    df['TimeOnly'] = df['Time'].dt.time

    # Standard time grid (06:00 to 21:00, 15-min intervals)
    standard_times = pd.date_range("06:00", "21:00", freq="15min").time

    # Reindex each day to match time grid
    def reindex_day(day_df, day):
        day_df = day_df.set_index('TimeOnly').reindex(standard_times)
        day_df['Date'] = day
        return day_df.reset_index().rename(columns={'index': 'TimeOnly'})

    all_days = df['Date'].unique()
    cleaned = [reindex_day(df[df['Date'] == day].copy(), day) for day in all_days]

    # Combine and recreate full datetime
    df_cleaned = pd.concat(cleaned, ignore_index=True)
    df_cleaned['Time'] = pd.to_datetime(df_cleaned['Date'].astype(str) + ' ' + df_cleaned['TimeOnly'].astype(str))
    return df_cleaned

# Apply to each dataset
mv = clean_timestamp(mv)
ov = clean_timestamp(ov)
p = clean_timestamp(p)

# Quick preview



In [ ]:
import re

def clean_panel_columns(df):
    # Leave Time-related columns as they are
    fixed_cols = ['Time', 'Date', 'TimeOnly']
    cleaned_cols = {}

    for col in df.columns:
        if col in fixed_cols:
            cleaned_cols[col] = col
        else:
            # Extract the core panel ID, e.g., "OP3.1.1 V (V)" -> "3.1.1"
            match = re.search(r'(\d+\.\d+\.\d+)', col)
            if match:
                cleaned_cols[col] = match.group(1)
            else:
                cleaned_cols[col] = col  # fallback, if it doesn't match

    df = df.rename(columns=cleaned_cols)
    return df

# Apply to all 3 datasets
mv = clean_panel_columns(mv)
ov = clean_panel_columns(ov)
p = clean_panel_columns(p)

# Preview
print("✅ Cleaned MV Columns:", mv.columns[3:8].tolist())


✅ Cleaned MV Columns: ['3.1.2', '3.1.3', '3.1.4', '3.1.5', '3.1.6']


In [ ]:
# Step 3 (fixed): Drop extra columns before melting

# Keep only Time + panel columns
mv_melt = mv.drop(columns=['Date', 'TimeOnly'])
ov_melt = ov.drop(columns=['Date', 'TimeOnly'])
p_melt  = p.drop(columns=['Date', 'TimeOnly'])

# Melt into long format
mv_long = mv_melt.melt(id_vars='Time', var_name='Panel', value_name='MV')
ov_long = ov_melt.melt(id_vars='Time', var_name='Panel', value_name='OV')
p_long  = p_melt.melt(id_vars='Time', var_name='Panel', value_name='P')

# Merge them on Time + Panel
merged_df = mv_long.merge(ov_long, on=['Time', 'Panel'])
merged_df = merged_df.merge(p_long, on=['Time', 'Panel'])

# Preview
merged_df.head()


,Time,Panel,MV,OV,P
0,2024-11-01 06:00:00,3.1.1,NaN,NaN,NaN
1,2024-11-01 06:15:00,3.1.1,NaN,NaN,NaN
2,2024-11-01 06:30:00,3.1.1,NaN,NaN,NaN
3,2024-11-01 06:45:00,3.1.1,NaN,NaN,NaN
4,2024-11-01 07:00:00,3.1.1,NaN,NaN,NaN


In [ ]:
# Step 4: Interpolate missing MV, OV, P — and track missingness per panel

# First, log the number of NaNs per panel before interpolation
nan_summary = merged_df.groupby('Panel')[['MV', 'OV', 'P']].apply(lambda x: x.isna().sum())
nan_summary.columns = ['MV_NaNs', 'OV_NaNs', 'P_NaNs']
nan_summary['Total_NaNs'] = nan_summary.sum(axis=1)

print("📝 Missing value summary before interpolation:")
display(nan_summary.sort_values('Total_NaNs', ascending=False))

# Sort for clean interpolation
merged_df = merged_df.sort_values(['Panel', 'Time'])

# Interpolate within each panel group
merged_df[['MV', 'OV', 'P']] = merged_df.groupby('Panel')[['MV', 'OV', 'P']].transform(lambda x: x.interpolate())

# Optional: fill any remaining edge NaNs (start/end) with 0s
merged_df = merged_df.fillna(0)

# Preview final cleaned version
merged_df.head()


📝 Missing value summary before interpolation:


,MV_NaNs,OV_NaNs,P_NaNs,Total_NaNs
Panel,,,,
3.1.1,1016,1017,1016,3049
3.1.3,1001,1002,1001,3004
3.1.21,963,964,963,2890
3.1.12,953,955,953,2861
3.1.11,925,927,925,2777
3.1.24,925,927,925,2777
3.1.8,921,923,921,2765
3.1.10,919,921,919,2759
3.1.14,917,919,917,2753


,Time,Panel,MV,OV,P
0,2024-11-01 06:00:00,3.1.1,0.0,0.0,0.0
1,2024-11-01 06:15:00,3.1.1,0.0,0.0,0.0
2,2024-11-01 06:30:00,3.1.1,0.0,0.0,0.0
3,2024-11-01 06:45:00,3.1.1,0.0,0.0,0.0
4,2024-11-01 07:00:00,3.1.1,0.0,0.0,0.0


In [ ]:
# Create panel-level missing count feature
panel_missing = nan_summary['Total_NaNs'].reset_index()
panel_missing.columns = ['Panel', 'Missing_Count']

# Merge into the merged_df
merged_df = merged_df.merge(panel_missing, on='Panel', how='left')




In [ ]:
# Check the shape of your merged_df
print("Shape:", merged_df.shape)

# Preview the first few rows
merged_df.head()


Shape: (45750, 6)


,Time,Panel,MV,OV,P,Missing_Count
0,2024-11-01 06:00:00,3.1.1,0.0,0.0,0.0,3049
1,2024-11-01 06:15:00,3.1.1,0.0,0.0,0.0,3049
2,2024-11-01 06:30:00,3.1.1,0.0,0.0,0.0,3049
3,2024-11-01 06:45:00,3.1.1,0.0,0.0,0.0,3049
4,2024-11-01 07:00:00,3.1.1,0.0,0.0,0.0,3049


In [ ]:
# Step: Compute summary stats per panel
panel_stats = merged_df.groupby('Panel').agg({
    'MV': ['mean', 'std', 'min', 'max'],
    'OV': ['mean', 'std', 'min', 'max'],
    'P':  ['mean', 'std', 'min', 'max']
})

# Flatten multi-level column names
panel_stats.columns = ['_'.join(col).lower() for col in panel_stats.columns]
panel_stats = panel_stats.reset_index()

# Merge with existing merged_df
merged_df = merged_df.merge(panel_stats, on='Panel', how='left')

# Preview updated structure
merged_df.head()


,Time,Panel,MV,OV,P,Missing_Count,mv_mean,mv_std,mv_min,mv_max,ov_mean,ov_std,ov_min,ov_max,p_mean,p_std,p_min,p_max
0,2024-11-01 06:00:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469
1,2024-11-01 06:15:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469
2,2024-11-01 06:30:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469
3,2024-11-01 06:45:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469
4,2024-11-01 07:00:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469


In [ ]:
# Define the known faulty panels (from supervisor + top missing)
faulty_panels = ['3.1.1', '3.1.3', '3.1.10', '3.1.12', '3.1.21', '3.1.5']  # swap/add more if needed

# Add a binary label to each row in merged_df
merged_df['Label'] = merged_df['Panel'].apply(lambda p: 1 if p in faulty_panels else 0)

# Preview
print(merged_df[['Time', 'Panel', 'Label']].head())


                 Time  Panel  Label
0 2024-11-01 06:00:00  3.1.1      1
1 2024-11-01 06:15:00  3.1.1      1
2 2024-11-01 06:30:00  3.1.1      1
3 2024-11-01 06:45:00  3.1.1      1
4 2024-11-01 07:00:00  3.1.1      1


In [ ]:
# Ensure Time column is parsed as datetime
merged_df['Time'] = pd.to_datetime(merged_df['Time'])

# Create a new column for date only (this helps us split by day)
merged_df['Date'] = merged_df['Time'].dt.date


In [ ]:
merged_df.head()


,Time,Panel,MV,OV,P,Missing_Count,mv_mean,mv_std,mv_min,mv_max,ov_mean,ov_std,ov_min,ov_max,p_mean,p_std,p_min,p_max,Label,Date
0,2024-11-01 06:00:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469,1,2024-11-01
1,2024-11-01 06:15:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469,1,2024-11-01
2,2024-11-01 06:30:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469,1,2024-11-01
3,2024-11-01 06:45:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469,1,2024-11-01
4,2024-11-01 07:00:00,3.1.1,0.0,0.0,0.0,3049,30.604779,10.6489,0.0,55.5,15.515973,6.459922,0.0,44.0,11.781929,20.715757,0.0,126.0469,1,2024-11-01


In [ ]:
group_sizes = merged_df.groupby(['Panel', 'Date']).size()
print("✅ How many (Panel, Date) groups:", len(group_sizes))
print("✅ Groups with 61 rows:", (group_sizes == 61).sum())
print("❌ Groups missing rows:", (group_sizes < 61).sum())


✅ How many (Panel, Date) groups: 750
✅ Groups with 61 rows: 750
❌ Groups missing rows: 0


In [ ]:
print("🔍 NaN counts:\n", merged_df[['MV', 'OV', 'P']].isna().sum())


🔍 NaN counts:
 MV    0
OV    0
P     0
dtype: int64


In [ ]:
import numpy as np

sequences = []
labels = []

# We'll go group-by-group: one (Panel, Date) at a time
for (panel, date), group in merged_df.groupby(['Panel', 'Date']):
    if group.shape[0] == 61:
        # Sort just in case
        group = group.sort_values(by='Time')

        # Extract MV, OV, P in order → shape (61, 3)
        sequence = group[['MV', 'OV', 'P']].to_numpy().T  # Transpose → shape (3, 61)
        label = group['Label'].iloc[0]  # Same label for the whole group

        sequences.append(sequence)
        labels.append(label)

# Convert to final tensors
X = np.stack(sequences)  # shape → (750, 3, 61)
y = np.array(labels)     # shape → (750,)

# Final check
print("✅ CNN Input X shape:", X.shape)
print("✅ Labels y shape:", y.shape)
print("📊 Label distribution:", np.bincount(y))


✅ CNN Input X shape: (750, 3, 61)
✅ Labels y shape: (750,)
📊 Label distribution: [570 180]


In [ ]:
# Save to disk in Colab
np.save("X_november.npy", X)
np.save("y_november.npy", y)


from google.colab import files

# Download both files
files.download("X_november.npy")
files.download("y_november.npy")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>